In [6]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import accuracy_score, classification_report, f1_score, roc_curve, auc, confusion_matrix, balanced_accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import gc
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
import pandas as pd
import shutil
from collections import defaultdict

# Set GPU memory growth to avoid allocating all memory at once
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Set GPU memory growth
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Found {len(gpus)} GPU devices and enabled memory growth")
    except RuntimeError as e:
        print(f"GPU setup error: {e}")


SEED = 3888
def set_seed(seed=3888):
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
set_seed(SEED)

# Define image size and batch size
IMG_SIZE = 224 

Found 1 GPU devices and enabled memory growth


In [ ]:
type_to_label_map = {
    'Non-Tumor': 0,
    'Tumor': 1,
    'Empty': 2
}

def preprocess_image(img_path, target_size=(224, 224)):
    img = Image.open(img_path)
    img = img.resize(target_size)
    img_array = np.array(img, dtype=np.float32)
    img_array = img_array / 255.0
    
    return img_array

def load_test_data(dir, quadrant, target_size=(224, 224)):
    """Load test data from a directory without separating by quadrants"""
    X_id = []
    X_test = []
    y_test = []
    

    quadrant_path = os.path.join(dir, quadrant)
    
    # Go through each cell type in the quadrant
    for cell_type in os.listdir(quadrant_path):
        cell_type_path = os.path.join(quadrant_path, cell_type)
        if os.path.isdir(cell_type_path):
            for filename in os.listdir(cell_type_path):
                if filename.endswith(".png") or filename.endswith(".jpg"):
                    img_path = os.path.join(cell_type_path, filename)
                    img = preprocess_image(img_path, target_size=target_size)
                    
                    parts = filename.replace(".png", "").replace(".jpg", "").split("_")
                    grid_id = f"{parts[1]}_{parts[2]}"

                    X_id.append(grid_id)
                    X_test.append(img)
                    y_test.append(type_to_label_map[cell_type])

    return X_id, np.array(X_test), np.array(y_test)

In [ ]:
def evaluate_model(model_dir, X_test, y_test):
    model = load_model(model_dir)
    
    # One-hot encode the test labels
    y_test_cat = to_categorical(y_test, num_classes=3)
    
    print(f"Test set shape: {X_test.shape}")
    print(f"Label distribution in test set: {np.unique(y_test, return_counts=True)}")
    
    # Make predictions
    y_prob = model.predict(X_test)
    y_pred = np.argmax(y_prob, axis=1)
    
    results = {
        'y_true': y_test,
        'y_pred': y_pred,
    }
    
    return results

In [25]:
results = []
for quadrant in ['Q1', 'Q2', 'Q3', 'Q4']:
    print(f"Evaluating model for {quadrant}")
    X_ids, X_test, y_test = load_test_data("projectdata/images/100_stratified4fold_1000per_seed3888", quadrant)
    result = evaluate_model(f"inceptionv3_test_on_{quadrant}.h5", X_test, y_test)
    results.append(result)

Evaluating model for Q1
Test set shape: (1050, 224, 224, 3)
Label distribution in test set: (array([0, 1, 2]), array([503, 447, 100], dtype=int64))
33/33 [==============================] - 4s 76ms/step
Evaluating model for Q2
Test set shape: (990, 224, 224, 3)
Label distribution in test set: (array([0, 1, 2]), array([503, 387, 100], dtype=int64))
31/31 [==============================] - 3s 52ms/step
Evaluating model for Q3
Test set shape: (1103, 224, 224, 3)
Label distribution in test set: (array([0, 1, 2]), array([503, 500, 100], dtype=int64))
35/35 [==============================] - 3s 54ms/step
Evaluating model for Q4
Test set shape: (1103, 224, 224, 3)
Label distribution in test set: (array([0, 1, 2]), array([503, 500, 100], dtype=int64))
35/35 [==============================] - 3s 49ms/step


In [ ]:
from scipy.stats import chisquare
import numpy as np

def chi_square_test(y_true, y_pred, labels=[0, 1, 2]):
    true_counts = np.array([np.sum(y_true == label) for label in labels])
    pred_counts = np.array([np.sum(y_pred == label) for label in labels])
    
    print("True label counts:", true_counts)
    print("Predicted label counts:", pred_counts)
    
    chi2_stat, p_value = chisquare(f_obs=pred_counts, f_exp=true_counts)
    
    print(f"Chi-square statistic = {chi2_stat:.4f}, p-value = {p_value:.4f}")
    
    return chi2_stat, p_value


i = 1
for result in results:
    print(f"Chi-square test for Q{i}")
    chi_square_test(result['y_true'], result['y_pred'])
    print("\n")
    i += 1

Chi-square test for Q1
True label counts: [503 447 100]
Predicted label counts: [425 525 100]
Chi-square statistic = 25.7062, p-value = 0.0000


Chi-square test for Q2
True label counts: [503 387 100]
Predicted label counts: [430 456 104]
Chi-square statistic = 23.0568, p-value = 0.0000


Chi-square test for Q3
True label counts: [503 500 100]
Predicted label counts: [395 620  88]
Chi-square statistic = 53.4289, p-value = 0.0000


Chi-square test for Q4
True label counts: [503 500 100]
Predicted label counts: [470 515 118]
Chi-square statistic = 5.8550, p-value = 0.0535


